# GEMdataset

> This package will hold the GxExM dataset.

In [ ]:
#| default_exp GEMdataset

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#| export
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

from pathlib import Path
from sklearn.preprocessing import StandardScaler,MinMaxScaler


In [ ]:
#| export
def clip_weatherdays(weather_table,days_of_year = 300):
    """
    Input : Pandas Table w/ weather data
    Output : Pandas Table w/ # of days for each environment clipped (from beginning of year.
    """
    to_remove = []
    for i in set(weather_table['Env']):
        for x in (weather_table.loc[weather_table['Env'] == i].index[days_of_year:]):
            to_remove.append(x)
    return weather_table.drop(index=to_remove)

In [ ]:
#| export
def collect_snps(method):
    """
    Input
        method: a Path(PosixPath) to directory containing npy arrays for each chr
    Output
        tuple(hybrid ids, snp matrix)
    """
    for c,chr in enumerate(method.iterdir()):
        if c == 0:
            strains,snp_data = np.load(chr,allow_pickle=True)
        else:
            strains, snps = np.load(chr, allow_pickle=True)
            snp_data = np.vstack((snp_data,snps))

    return strains,snp_data

In [ ]:
#| export
def prep_gem_data(
    trait_csv,
    weather_csv,
    snp_folder
):
    phenotype = pd.read_csv(trait_csv)
    weather = pd.read_csv(weather_csv)
    genotype = collect_snps(Path(snp_folder))
    
    print(f'{phenotype.shape[0]} plots with {phenotype.shape[1]} features')
    print(f'{weather.shape[0]} daily weather measurements with {weather.shape[1]} features')
    print(f'{genotype[0].shape[0]} hybrids')
    
    #clip days per year
    weather = clip_weatherdays(weather)
    #add year column
    weather['Year'] = [x.split('_')[1] for x in weather['Env']] #Store Year in a new column
    
    #removes yield data where no weather data
    setYield = set(phenotype['Env'])
    setWeather = set(weather['Env'])
    only_yield = setYield - setWeather
    only_weather = setWeather - setYield
    phenotype = phenotype.iloc[[x not in only_yield for x in phenotype['Env']],:]
    #removes yield data where no genotype data
    setSNP = set(genotype[0])
    setYield = set(phenotype['Hybrid'])
    only_yield = setYield - setSNP
    phenotype = phenotype.iloc[[x not in only_yield for x in phenotype['Hybrid']],:]
    
    weather = weather.reset_index()
    phenotype = phenotype.reset_index()
    
    return phenotype, genotype, weather

In [ ]:
#| export
class WT():
    """
    A class which will hold the weather data for the entire dataset for training purposes
    
    init
        weather_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, weather_data, testYear):
        
        self.Te = weather_data.iloc[([str(testYear) in x for x in weather_data['Year']]),:].reset_index()
        self.Tr = weather_data.iloc[([str(testYear) not in x for x in weather_data['Year']]),:].reset_index()
            
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

        self.make_array(self.Tr)
        self.make_array(self.Te,False)
            
    def setup_scaler(self):
        ss = MinMaxScaler()
        ss.fit(self.Tr.select_dtypes('float'))
        self.scaler = ss
            
    def scale_data(self, df):
        fd = df.select_dtypes('float')
        fs = self.scaler.transform(fd)
        df[fd.columns] = fs

    def make_array(self, df,train = True):
      """ convert from pandas table to numpy array"""
      for c,i in enumerate(set(df['Env'])):
        env_weather = np.array(df[df['Env'] == i].iloc[:,4:-1])
        #print(env_weather.shape)
        if c == 0:
          env_order = list([i])
          weather_array =   np.array(df[df['Env'] == i].iloc[:,4:-1])
          weather_array = np.expand_dims(weather_array,axis=0)
        else:
          weather_array = np.vstack((weather_array, env_weather[None,:,:]))
          env_order.append(i)

        if train:
          self.Tr = (np.array(env_order), np.array(weather_array))
        else:
          self.Te = (np.array(env_order), np.array(weather_array))

In [ ]:
#| export
class YT():
    """
    A class which will hold the yield data for the entire dataset for training purposes
    
    init
        yield_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, yield_data, testYear):

        self.Te = yield_data.iloc[([str(testYear) in x for x in yield_data['Env']]),:].reset_index()
        self.Tr = yield_data.iloc[([str(testYear) not in x for x in yield_data['Env']]),:].reset_index()

        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

    def setup_scaler(self):
        ss = StandardScaler()
        ss.fit(np.array(self.Tr['Yield_Mg_ha']).reshape(-1,1))
        self.scaler = ss

    def scale_data(self,df):
        ya = np.array(df['Yield_Mg_ha']).reshape(-1,1)
        ys = self.scaler.transform(ya)
        df['scaled_yield'] = ys

    def plot_yields(self):

        plt.hist(self.Tr['scaled_yield'],density=True, label='Train',alpha=.5,bins=50)
        plt.hist(self.Te['scaled_yield'],density=True, label='Test',alpha=.5,bins=50)
        plt.legend()
        plt.show()

In [ ]:
#| export
class ST():
    """
    A class which will hold the secondary trait data for the entire dataset for pre-training purposes
    
    init
        yield_data -> pandas table
        testYear -> e.g. 2019. this will set all data from a given year as the Test Set
    """
    def __init__(self, yield_data, testYear):

        self.Te = yield_data.iloc[([str(testYear) in x for x in yield_data['Env']]),:].reset_index()
        self.Tr = yield_data.iloc[([str(testYear) not in x for x in yield_data['Env']]),:].reset_index()

        self.secondary_traits = [
                'Stand_Count_plants',
                'Pollen_DAP_days',
                'Silk_DAP_days',
                'Plant_Height_cm',
                'Ear_Height_cm',
                #'Root_Lodging_plants',
                #'Stalk_Lodging_plants',
                'Twt_kg_m3',
                'Yield_Mg_ha',
                #'Date_Harvested'
                ]
        
        self.setup_scaler()
        self.scale_data(self.Tr)
        self.scale_data(self.Te)

        self.make_arrays(self.Tr)
        self.make_arrays(self.Te, False)
    def setup_scaler(self):
        ss = StandardScaler()
        ss.fit(np.array(self.Tr[self.secondary_traits]))
        self.scaler = ss

    def scale_data(self,df):
        scaled_secondary = self.scaler.transform(np.array(df[self.secondary_traits]))
        for c,i in enumerate(self.secondary_traits):
            #print(i)
            df[i] = scaled_secondary[:,c]
    
    def plot_yields(self):
        for i in self.secondary_traits:
            plt.hist(self.Tr[i],density=True, label='Train',alpha=.5,bins=50)
            plt.hist(self.Te[i],density=True, label='Test',alpha=.5,bins=50)
            plt.legend()
            plt.title(i)
            plt.show()

    def make_arrays(self,df,train=True):
      df = np.array(df[self.secondary_traits]), np.array(df['Env']) , np.array(df['Hybrid']), np.array(df['Date_Planted'])
      if train:
        self.Tr = df
      else:
        self.Te=df

In [ ]:
#| export
class GemDataset():
    """
    Pytorch Dataset which can be used with dataloaders for simple batching during training loops
    """
    def __init__(self,W,Y,G):
        self.W = W
        self.SNP = G
        self.Y = Y
        
    def __len__(self): return self.Y[0].shape[0]

    def __getitem__(self,idx):
      y = self.Y[0][idx]
      e = self.Y[1][idx]
      h = self.Y[2][idx]
      d = self.Y[3][idx]

      #weather
      w = self.W[1][np.where(self.W[0] == e)[0][0]]

      #snp
      g = self.SNP[1][:,np.where(self.SNP[0] == h)[0][0]]
      return y,g,w

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()